In [24]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
blok_size = 8 #блок - это длинна секции вектора, а батч - несколько блоков
batch_size = 4

cpu


In [1]:
wizard_of_oz_book_data_path = 'oz_book\wizard_of_oz.txt'

In [5]:
with open(wizard_of_oz_book_data_path, 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [15]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# encoded_hello = encode('hello')
# decoded_hello = decode(encoded_hello)
# print(decoded_hello)

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [23]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - blok_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+blok_size] for i in ix])
    y = torch.stack([data[i+1:i+blok_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
#print(x.shape)
print(x)
print('targets:')
print(y)

tensor([ 67706,  38842, 123936,  75835])
inputs:
tensor([[ 1, 73, 61, 58, 78,  1, 61, 54],
        [73,  1, 54, 67, 57,  0, 54,  1],
        [66, 58,  1, 59, 71, 58, 58,  1],
        [73, 61, 58, 71,  1, 54, 67, 57]])
targets:
tensor([[73, 61, 58, 78,  1, 61, 54, 75],
        [ 1, 54, 67, 57,  0, 54,  1, 66],
        [58,  1, 59, 71, 58, 58,  1, 59],
        [61, 58, 71,  1, 54, 67, 57,  1]])


In [17]:
x = train_data[:blok_size]
y = train_data[1:blok_size+1]

for t in range(blok_size):
    context = x[:t+1]
    target = y[t]
    print(f'Когда на входе {context} на выходе (цель) {target}')

Когда на входе tensor([80]) на выходе (цель) 1
Когда на входе tensor([80,  1]) на выходе (цель) 1
Когда на входе tensor([80,  1,  1]) на выходе (цель) 28
Когда на входе tensor([80,  1,  1, 28]) на выходе (цель) 39
Когда на входе tensor([80,  1,  1, 28, 39]) на выходе (цель) 42
Когда на входе tensor([80,  1,  1, 28, 39, 42]) на выходе (цель) 39
Когда на входе tensor([80,  1,  1, 28, 39, 42, 39]) на выходе (цель) 44
Когда на входе tensor([80,  1,  1, 28, 39, 42, 39, 44]) на выходе (цель) 32


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embending_table == nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets):
        logits = self.token_embending_table(index)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        
        return logits